In [ ]:
import scanpy as sc
import numpy as np
import scprep
from anndata import AnnData
import phate
from phate.cluster import kmeans
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import sys
sys.path.append('..')
from dhg import Hypergraph
from hypgs.models.hsn_pyg import HSN
from hypgs.utils.data import HGDataset
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from torch.utils.data import random_split
from pytorch_lightning.loggers import TensorBoardLogger
from hypgs import DATA_DIR
import anndata
import os
import torch
import numpy as np
import phate
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
import scanpy as sc
from sklearn.neighbors import kneighbors_graph
import random 
from torch_geometric.utils.convert import from_networkx
from importlib import reload
from hypgs.models.hsn_pyg import HyperScatteringModule
from torch_geometric.utils.convert import from_networkx
from einops import rearrange
from scipy.spatial import ConvexHull
from hypgs.utils.plot import plot_wavelets, get_hyperedge_pos_df, compute_enlarged_hulls, plot_hulls
from hypgs.utils.hypergraph_utils import get_cliques_planar

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

In [ ]:
adata_subset = sc.read_h5ad("merfish_H20.33.001.CX28.MTG.02.007.1.02.02.h5ad", backed='r')

In [ ]:
def get_plots(num_hops, K=10):
    sparseA = kneighbors_graph(adata_subset.obsm['X_selected_cell_spatial_tiled'], K, mode='connectivity', include_self=False)
    A = sparseA.todense()
    G = nx.from_numpy_array(A)
    # X_cmbn = np.concatenate((adata_subset[:,:].X.toarray(), adata_subset.obsm['X_phate']), axis=1)
    X_cmbn = adata_subset[:,:].X.toarray()
    data = from_networkx(G)
    data.x = torch.tensor(X_cmbn)
    original_dataset = [data]
    to_hg_func = lambda g: Hypergraph.from_graph_kHop(g, num_hops)
    dataset = HGDataset(original_dataset, to_hg_func)
    coordinates = adata_subset.obsm['X_selected_cell_spatial_tiled']
    eidf = get_hyperedge_pos_df(dataset[0], coordinates)
    enlarged_hulls = compute_enlarged_hulls(eidf)
    model = HyperScatteringModule(in_channels=1, # doesn't matter here.
              trainable_laziness = False,
              trainable_scales = False, 
              activation = None, # just get one layer of wavelet transform 
              fixed_weights=True, 
              normalize='right', 
              reshape=False,
        ).to(device)
    init_node_sig = torch.from_numpy(X_cmbn[:, 1].reshape(-1, 1)).to(device)
    init_he_sig = torch.zeros(dataset[0].edge_attr.shape[0], 1, device=device)
    heidx = dataset[0].edge_index.to(device)
    s_nodes, s_edges = model(init_node_sig, heidx, hyperedge_attr = init_he_sig)
    node_feats = s_nodes[0,:,:,0].detach().cpu().numpy().T
    edge_feats = s_edges[0,:,:,0].detach().cpu().numpy().T
    plot_wavelets(node_feats, edge_feats, coordinates, enlarged_hulls, title=f'Marker, k={K}, n_hops={num_hops}', alpha=0.2)
    plt.show()
    scprep.plot.scatter2d(coordinates, c=init_node_sig.cpu().numpy(), cmap='viridis')
    plt.show()
    np.random.seed(32)
    one_pos = np.random.choice(X_cmbn.shape[0], 10)
    dirac_sig = torch.zeros((X_cmbn.shape[0], 1))
    dirac_sig[one_pos, :] = 1
    init_node_sig = dirac_sig.to(device)
    init_he_sig = torch.zeros(dataset[0].edge_attr.shape[0], 1, device=device)
    heidx = dataset[0].edge_index.to(device)
    s_nodes, s_edges = model(init_node_sig, heidx, hyperedge_attr = init_he_sig)
    node_feats = s_nodes[0,:,:,0].detach().cpu().numpy().T
    edge_feats = s_edges[0,:,:,0].detach().cpu().numpy().T
    plot_wavelets(node_feats, edge_feats, coordinates, enlarged_hulls, title=f'Dirac, k={K}, n_hops={num_hops}', alpha=0.2)
    plt.show()
    scprep.plot.scatter2d(coordinates, c=init_node_sig.cpu().numpy(), cmap='viridis')
    plt.show()
    return dataset

In [ ]:
for k in [3, 5, 10, 15]:
    for h in [1, 2, 3, 4]:
        get_plots(h, k)

In [ ]:
# data_from_clique = get_cliques_planar(data)

In [ ]:
# data_from_clique

In [ ]:
# data_from_clique.three_cliques